In [2]:
import pandas as pd
import pickle
# from gensim.models import fasttext
from tqdm import tqdm
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

from sklearn.preprocessing import LabelEncoder

from torch.utils.data import Dataset, DataLoader

from torchmetrics import F1Score

import gensim
from gensim.models import fasttext

ft_path = '.' # fast text가 저장된 경로
ft_model = gensim.models.fasttext.load_facebook_model( ft_path + '/wiki.ko.bin')

C:\Users\user\anaconda3\envs\DDECC\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\user\anaconda3\envs\DDECC\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")
C:\Users\user\anaconda3\envs\DDECC\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\user\anaconda3\envs\DDECC\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


## 데이터 불러오기

In [7]:
path = './ETRI_ERC/dataset/KEMDy20_v1_1/new/text'

# 저장한 데이터 토큰 피클 파일 불러오기
with open(path + '/token_NonNP.pkl', 'rb') as f:
    text_data = pickle.load(f)

text_data['token_len'] = [len(x) for x in text_data['token']]

# 다중 레이블 처리한 훈련 데이터셋 가져오기
with open(path+'/train_origin.pkl', 'rb') as f:
    train_data = pickle.load(f)

# 다중 레이블 처리한 테스트 데이터셋 가져오기
with open(path+'/test_origin.pkl', 'rb') as f:
    test_data = pickle.load(f)

train_data

,segment_id,sess,script,seconds,emotion,emotion_id
0,Sess01_script01_User002M_001,1,01,8.059001,neutral,4
1,Sess01_script01_User002M_002,1,01,11.697002,neutral,4
2,Sess01_script01_User002M_003,1,01,8.223999,neutral,4
3,Sess01_script01_User002M_004,1,01,11.411998,neutral,4
4,Sess01_script01_User001F_001,1,01,2.086000,neutral,4
...,...,...,...,...,...,...
10135,Sess40_script06_User079F_043,40,06,5.493020,neutral,4
10136,Sess40_script06_User079F_044,40,06,2.655000,neutral,4
10137,Sess40_script06_User079F_045,40,06,6.573000,neutral,4
10138,Sess40_script06_User079F_046,40,06,2.692000,neutral,4


## 필요 데이터 추출

In [8]:
# 필요한 컬럼만 남겨두기
train_data = train_data.drop(['seconds', 'sess', 'script'], axis=1)
test_data = test_data.drop(['seconds', 'sess', 'script'], axis=1)

# 텍스트에서 필요한 컬럼만 남겨두기 및 컬럼 이름 맞추기
text_data = text_data[['segment_id', 'token', 'token_len']]
#text_data = text_data.rename(columns={'Segment ID':'segment_id'})

In [9]:
# 훈련 및 테스트 데이터에 텍스트 데이터 붙이기
train = pd.merge(train_data, text_data, how='left', on=['segment_id'])
test = pd.merge(test_data, text_data, how='left', on=['segment_id'])

In [10]:
train.head(10)

,segment_id,emotion,emotion_id,token,token_len
0,Sess01_script01_User002M_001,neutral,4,"[는, 어, 어머니, 엄마, 가, 이제, 생일, 때, 마다, 이제, 미역국, 도, ...",20
1,Sess01_script01_User002M_002,neutral,4,"[전, 에, 도, 말, 했, 지만, 그래서, 미역국, 이랑, 볶음, 김치, 막, 해...",41
2,Sess01_script01_User002M_003,neutral,4,"[어, 응, 되게, 엄, 집, 앞, 에, 그, 조그만, 한, 가게, 로, 있, 었,...",24
3,Sess01_script01_User002M_004,neutral,4,"[서, 엄마, 랑, 둘, 이, 생일, 파티, 그냥, 간단, 하, 게, 하, 는, 경...",43
4,Sess01_script01_User001F_001,neutral,4,"[아, 친구, 들, 도]",4
5,Sess01_script01_User002M_005,neutral,4,"[응, 응, 그래서, 생일, 때, 그렇, 게, 했, 던, 경험, 이, 있, 는데, ...",36
6,Sess01_script01_User002M_006,neutral,4,"[그래서, 이제, 막, 지역, 돌아다니, 면서, 캔, 모아, 빙수, 있, 는, 가게...",23
7,Sess01_script01_User002M_007,happy,3,"[어, 그래서, 그런, 가게, 들, 갈, 때, 면, 맨날, 엄마, 생각, 이, 되게...",16
8,Sess01_script01_User002M_008,happy,3,"[그래서, 일부러, 보이, 면, 일부러, 가, 기, 도, 해]",9
9,Sess01_script01_User002M_009,happy,3,"[그때, 먹, 었, 던, 빙수, 맛, 이, 그대로, 인데, 되게, 맛있, 었, 거든]",13


## 데이터 증강

In [11]:
import random
import pandas as pd
import tqdm

# hyperparameter
n_a = 0.3 # 문장 내에서 변환할 단어의 비율
del_p = 0.2 # 문장 내에서 단어를 삭제할 확률
top_sim = 1 # 동의어 대체에 사용할 fasttext의 유사 단어 개수

# 랜덤한 단어를 선택하여 랜덤한 동의어로 변환
def SR(tokens, a):
    num_modify = int(len(tokens)*a) # 변환할 단어의 개수
    modify_idx_list = list(range(0, len(tokens)))
    random.shuffle(modify_idx_list)
    modify_idx_list = modify_idx_list[:num_modify] # 변환할 토큰의 인덱스
    
    for mod_idx in modify_idx_list:
        rand_rank = random.randrange(0, top_sim) # fasttext의 상위 5개 동의어 중 하나로 변환 
        # 현재 단어와 가장 유사한 단어를 가져옴
        synonym = ft_model.wv.similar_by_word(tokens[mod_idx], topn=top_sim)[rand_rank][0]
        tokens[mod_idx] = synonym
    
    return tokens

# 랜덤한 단어를 선택해서 랜덤한 동의어로 변환한 후 랜덤한 위치에 넣음
def RI(tokens, a):
    num_modify = int(len(tokens)*a) # 변환할 단어의 개수
    
    for i in range(num_modify):
        mod_idx = random.randrange(0, len(tokens))
        rand_rank = random.randrange(0, top_sim)
        
        # 현재 단어와 가장 유사한 단어를 가져옴
        synonym = ft_model.wv.similar_by_word(tokens[mod_idx], topn=top_sim)[rand_rank][0]
        tokens[mod_idx] = synonym
        
    return tokens

# 랜덤한 단어 두 개를 뽑아 서로의 위치를 바꿈
def RS(tokens, a):
    num_modify = int(len(tokens)*a) # 변환할 단어의 개수
    
    for i in range(num_modify):
        # 임의의 두 개의 단어 뽑음
        modify_idx_list = list(range(0, len(tokens)))
        random.shuffle(modify_idx_list)
        a,b = modify_idx_list[:2]
        
        # 서로의 위치를 바꿈
        tokens[a], tokens[b] = tokens[b], tokens[a]
        
    return tokens

# p 확률로 단어를 삭제
def RD(tokens, p):
    # p 확률로 삭제할 단어 개수 계산
    num_del = int(len(tokens) * p)
    # num_del만큼 삭제할 단어의 인덱스 선택(pop하기 위해 정렬)
    del_idx_list = list(range(0, len(tokens)))
    random.shuffle(del_idx_list)
    del_idx_list = sorted(del_idx_list[:num_del], reverse=True)
    
    for del_idx in del_idx_list:
        tokens.pop(del_idx)
    
    return tokens

# augmentation
def augment_token(tokens, mode):
    augment_token = []
    if mode == 'SR':
        augment_token = SR(tokens.copy(), n_a)
    elif mode == 'RI':
        augment_token = RI(tokens.copy(), n_a)
    elif mode == 'RS':
        augment_token = RS(tokens.copy(), n_a)
    elif mode == 'RD':
        augment_token = RD(tokens.copy(), del_p)
    
    return augment_token

In [67]:
train[train['emotion_id']==4].sort_values('segment_id').head(10)

,segment_id,emotion,emotion_id,token,token_len
4,Sess01_script01_User001F_001,neutral,4,"[아, 친구, 들, 도]",4
12,Sess01_script01_User001F_002,neutral,4,"[는, 생일]",2
13,Sess01_script01_User001F_003,neutral,4,"[생일날, 이, 면은, 가, 고기, 되게, 좋, 아, 하, 니까, 엄마, 도, 그걸...",48
17,Sess01_script01_User001F_005,neutral,4,"[용돈, 으로, 주, 시, 고]",5
18,Sess01_script01_User001F_006,neutral,4,"[옛날, 에, 는, 가, 원, 하, 는, 게, 뭔지, 일단, 물, 어, 보, 시, ...",20
19,Sess01_script01_User001F_007,neutral,4,"[옷, 이, 라든가, 갖, 고, 싶, 은, 거, 있, 나라, 든가, 근데, 이제, ...",22
31,Sess01_script01_User001F_008,neutral,4,"[응, 도, 되게, 여행, 가, 는, 걸, 좋, 아, 하, 긴, 하, 는데, 가족,...",33
33,Sess01_script01_User001F_009,neutral,4,"[어, 단, 둘, 이]",4
35,Sess01_script01_User001F_010,neutral,4,"[그, 되게, 아빠, 나, 오빠, 가, 시간, 이, 안, 나, 거나, 꼭, 약속, ...",31
36,Sess01_script01_User001F_011,neutral,4,"[여행, 여행]",2


In [26]:
# 데이터 증강

x = {0: 90, 1: 200, 2: 260, 3: 10, 4: 1, 5: 100, 6: 90}

for i in [0,1,2,3,5,6]:
    a = train[train['emotion_id']== i].sample(frac=x[i], replace = True)
    if i < 1:
        aug_df = a
    else : aug_df = pd.concat([aug_df, a], axis=0)

In [30]:
len(train), len(aug_df), aug_df.head()

(10140,
 67900,
                         segment_id emotion  emotion_id  \
 4432  Sess19_script06_User038F_015   angry           0   
 1241  Sess08_script02_User016F_029   angry           0   
 4750  Sess21_script02_User042F_018   angry           0   
 1569  Sess09_script02_User018M_001   angry           0   
 8672  Sess35_script04_User069F_023   angry           0   
 
                                                   token  token_len  
 4432                     [어, 랑, 친해져서, 그, 친구, 랑, 친해, 지려]          8  
 1241  [아, 더, 어이없, 는, 거, 인사, 하, 면, 막상, 하, 면, 받, 아, 주,...         18  
 4750  [근데, 는, 어쨌든, 그, 말, 듣, 고서, 간, 건데, 오히려, 안, 온, 애,...         48  
 1569  [아, 근데, 너무, 좀, 화, 짜증, 나, 는, 게, 이게, 꼰대, 의, 기준, ...         23  
 8672  [는, 이런, 쓰레기, 같, 은, 사람, 소개, 시켜, 주, 냐고, 도, 몰랐, 다...         35  )

In [44]:
aug_df.head(5)

,segment_id,emotion,emotion_id,token,token_len
4432,Sess19_script06_User038F_015,angry,0,"[어, 랑, 친해져서, 그, 친구, 랑, 친해, 지려]",8
1241,Sess08_script02_User016F_029,angry,0,"[아, 더, 어이없, 는, 거, 인사, 하, 면, 막상, 하, 면, 받, 아, 주,...",18
4750,Sess21_script02_User042F_018,angry,0,"[근데, 는, 어쨌든, 그, 말, 듣, 고서, 간, 건데, 오히려, 안, 온, 애,...",48
1569,Sess09_script02_User018M_001,angry,0,"[아, 근데, 너무, 좀, 화, 짜증, 나, 는, 게, 이게, 꼰대, 의, 기준, ...",23
8672,Sess35_script04_User069F_023,angry,0,"[는, 이런, 쓰레기, 같, 은, 사람, 소개, 시켜, 주, 냐고, 도, 몰랐, 다...",35


In [46]:
# 3가지 데이터 증강 기법 랜덤 적용
from tqdm import tqdm
aug_result = []
aug_func = ['SR', 'RI', 'RS']
for tokens in tqdm(aug_df['token']):
    func_idx = random.randrange(0,3) # 사용할 증강 
    func_nm = aug_func[func_idx]
    
    augmented = augment_token(tokens, mode = func_nm)
    aug_result.append(augmented)

aug_df['augmented'] = aug_result

100%|██████████| 67900/67900 [1:42:04<00:00, 11.09it/s]  


In [47]:
# 데이터 저장
import pickle

aug_df.to_pickle(path+'augNonNP_df.pkl')

In [56]:
len(aug_df), len(train)

(67900, 10140)